# Summary so far 

# TripAdvisor API - Best Option for Data Collection but needs to carefully monitored
Two different searches 
- Location search - returns the attractions in the query zone "New York City" 
- Then to get details on said attractions we must then search the locationiD to show indepth details on each location
### NB V Important - 1200 calls a day is lowest billing setting and only 5000 free calls a month before the begin to bill. Can become a problem if not properly managed

# Ticketmaster API - returns event data in NYC - provides sufficient info
- Good for Time Dates and Venues of live events and fututre event planning
- Not much good for attractions or depth of detail
###  5000 calls a day and no need for card details - solid option

# Google Places API - also a great option but very tedious
- Great depth of detail on each attraction in single search 
- Major downside only returns 20 rows of attractions per search.
### In order to use would need to send off lots of searches while slightly moving the coordinates about in order to attempt to collect lots of attractions. Billing is free and easily managed

# Yelp API-
- Has good information available for free
- Daily Limit of 300 calls to API 
- Returned 127 rows when called 
- Not sure whether some rows are useful but are potentially (attendance may not be reliable) 



## TripAdvisor API - 5000 free calls a month - cannot set lower than 1200 calls a day limit so MUST BE CAREFUL TO NOT EXCEED LIMIT AS MY CARD IS ON ACCOUNT

In [5]:
import requests 

API_KEY = "XXX"

# Base URL for TripAdvisor API search
SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"

# Headers for the request
headers = {
    "accept": "application/json"
}

# Function to search for attractions in New York City
def search_attractions(query, limit=100):
    params = {
        "searchQuery": query,
        "language": "en",
        "key": API_KEY,
        "limit": limit  # Limit the number of results
    }
    
    response = requests.get(SEARCH_URL, headers=headers, params=params)
    response.raise_for_status()  # Check for HTTP errors
    data = response.json()
    
    # Extract basic information about each attraction
    attractions = [
        {
            "name": location.get("name"),
            "location_id": location.get("location_id"),
            "latitude": location.get("latitude"),
            "longitude": location.get("longitude"),
            "address": location.get("address_obj", {}).get("address_string"),
            "rating": location.get("rating"),
            "num_reviews": location.get("num_reviews")
        }
        for location in data.get("data", [])
    ]
    return attractions

# Fetch and print the first 100 attractions in New York City
attractions = search_attractions("New York City", 100)
for attraction in attractions:
    print(attraction)

{'name': 'New York City', 'location_id': '60763', 'latitude': None, 'longitude': None, 'address': 'New York City, NY', 'rating': None, 'num_reviews': None}
{'name': 'Motto by Hilton New York City Chelsea', 'location_id': '23448880', 'latitude': None, 'longitude': None, 'address': '113 West 24th Street, New York City, NY 10001', 'rating': None, 'num_reviews': None}
{'name': 'Arthouse Hotel New York City', 'location_id': '99288', 'latitude': None, 'longitude': None, 'address': '2178 Broadway at West 77th Street, New York City, NY 10024-6647', 'rating': None, 'num_reviews': None}
{'name': 'The Ludlow New York City', 'location_id': '6727310', 'latitude': None, 'longitude': None, 'address': '180 Ludlow St, New York City, NY 10002-1514', 'rating': None, 'num_reviews': None}
{'name': 'New York City Subway', 'location_id': '8130064', 'latitude': None, 'longitude': None, 'address': 'New York City, NY', 'rating': None, 'num_reviews': None}
{'name': 'Virgin Hotels New York City', 'location_id': '

In [8]:
url = "https://api.content.tripadvisor.com/api/v1/location/search?searchQuery=%22New%20York%20City%22&category=attractions&language=en&key=APIKEY"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{
  "data": [
    {
      "location_id": "8130064",
      "name": "New York City Subway",
      "address_obj": {
        "city": "New York City",
        "state": "New York",
        "country": "United States",
        "address_string": "New York City, NY"
      }
    },
    {
      "location_id": "555521",
      "name": "New York City Center",
      "address_obj": {
        "street1": "131 W 55th St",
        "street2": "Between 6th and 7th",
        "city": "New York City",
        "state": "New York",
        "country": "United States",
        "postalcode": "10019-5306",
        "address_string": "131 W 55th St Between 6th and 7th, New York City, NY 10019-5306"
      }
    },
    {
      "location_id": "555633",
      "name": "New York City Ballet",
      "address_obj": {
        "street1": "20 Lincoln Center Plz",
        "street2": "David H. Koch Theater, Lincoln Center for the Performing Arts complex",
        "city": "New York City",
        "state": "New York",
        "countr

In [9]:
len(response.text)

3875

In [14]:
import pandas as pd

API_KEY = "XXX"

# Base URL for TripAdvisor API search
SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"

# Headers for the request
headers = {
    "accept": "application/json"
}

# Function to search for attractions in New York City and return as a DataFrame
def search_attractions(query, category, total_results=100):
    attractions = []
    offset = 0
    limit = 50  # Adjust based on the API's max limit per request

    while len(attractions) < total_results:
        params = {
            "searchQuery": query,
            "category": category,
            "language": "en",
            "key": API_KEY,
            "limit": limit,
            "offset": offset
        }

        response = requests.get(SEARCH_URL, headers=headers, params=params)
        data = response.json()

        if "data" not in data or not data["data"]:
            break

        for location in data["data"]:
            address_obj = location.get("address_obj", {})
            attractions.append({
                "name": location.get("name"),
                "location_id": location.get("location_id"),
                "latitude": location.get("latitude"),
                "longitude": location.get("longitude"),
                "street": address_obj.get("street1"),
                "city": address_obj.get("city"),
                "state": address_obj.get("state"),
                "country": address_obj.get("country"),
                "postalcode": address_obj.get("postalcode"),
                "address": address_obj.get("address_string"),
                "rating": location.get("rating"),
                "num_reviews": location.get("num_reviews")
            })

        offset += limit

    return pd.DataFrame(attractions[:total_results])

# Fetch the first 100 attractions in New York City
df_attractions = search_attractions("New York City", "attractions", 100)

# Print the DataFrame
print(df_attractions)

                               name location_id latitude longitude  \
0              New York City Subway     8130064     None      None   
1              New York City Center      555521     None      None   
2              New York City Ballet      555633     None      None   
3       Resorts World New York City     2423159     None      None   
4                New York City Hall    10031098     None      None   
..                              ...         ...      ...       ...   
95                     The Ride NYC     1951181     None      None   
96        New York City Fire Museum      110276     None      None   
97  Secret Food Tours New York City    12423267     None      None   
98         Helicopter New York City     4005106     None      None   
99      Babylon Tours New York City    11891707     None      None   

                         street           city     state        country  \
0                          None  New York City  New York  United States   
1        

In [15]:
# Print the DataFrame
df_attractions

,name,location_id,latitude,longitude,street,city,state,country,postalcode,address,rating,num_reviews
0,New York City Subway,8130064,None,None,None,New York City,New York,United States,None,"New York City, NY",None,None
1,New York City Center,555521,None,None,131 W 55th St,New York City,New York,United States,10019-5306,"131 W 55th St Between 6th and 7th, New York Ci...",None,None
2,New York City Ballet,555633,None,None,20 Lincoln Center Plz,New York City,New York,United States,10023-6913,"20 Lincoln Center Plz David H. Koch Theater, L...",None,None
3,Resorts World New York City,2423159,None,None,110-00 Rockaway Blvd,Jamaica,New York,United States,11420-1009,"110-00 Rockaway Blvd, Jamaica, NY 11420-1009",None,None
4,New York City Hall,10031098,None,None,Broadway and Murray St,New York City,New York,United States,10007,"Broadway and Murray St City Hall Park, New Yor...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
95,The Ride NYC,1951181,None,None,W 42nd St,New York City,New York,United States,10036,"W 42nd St, New York City, NY 10036",None,None
96,New York City Fire Museum,110276,None,None,278 Spring Street,New York City,New York,United States,10013-1405,"278 Spring Street, New York City, NY 10013-1405",None,None
97,Secret Food Tours New York City,12423267,None,None,323 6th Avenue,New York City,New York,United States,10014-4403,"323 6th Avenue, New York City, NY 10014-4403",None,None
98,Helicopter New York City,4005106,None,None,Downtown Manhattan Heliport,New York City,New York,United States,10004,Downtown Manhattan Heliport Pier 6 and the Eas...,None,None


In [16]:
#using the location ID for 8130064 - New York City Subway - can get additional info - lat and long is included
url = "https://api.content.tripadvisor.com/api/v1/location/8130064/details?language=en&currency=USD&key=APIKEY"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{
  "location_id": "8130064",
  "name": "New York City Subway",
  "web_url": "https://www.tripadvisor.com/Attraction_Review-g60763-d8130064-Reviews-New_York_City_Subway-New_York_City_New_York.html?m=66827",
  "address_obj": {
    "city": "New York City",
    "state": "New York",
    "country": "United States",
    "address_string": "New York City, NY"
  },
  "ancestors": [
    {
      "level": "City",
      "name": "New York City",
      "location_id": "60763"
    },
    {
      "abbrv": "NY",
      "level": "State",
      "name": "New York",
      "location_id": "28953"
    },
    {
      "level": "Country",
      "name": "United States",
      "location_id": "191"
    }
  ],
  "latitude": "40.75588",
  "longitude": "-73.98698",
  "timezone": "America/New_York",
  "website": "https://new.mta.info/",
  "write_review": "https://www.tripadvisor.com/UserReview-g60763-d8130064-New_York_City_Subway-New_York_City_New_York.html?m=66827",
  "ranking_data": {
    "geo_location_id": "60763",
   

In [36]:
API_KEY = 'XXX'
location_ids = ['8130064', '12423267', '4005106']  # Example list of location IDs

data = []

for location_id in location_ids:
    url = f"https://api.content.tripadvisor.com/api/v1/location/{location_id}/details?language=en&currency=USD&key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        location_data = response.json()
        # Extract required information
        location_info = {
            "location_id": location_data["location_id"],
            "name": location_data["name"],
            "latitude": location_data["latitude"],
            "longitude": location_data["longitude"],
            "website": location_data.get("website", None),
            "rating": location_data.get("rating", None),
            "ranking_string": location_data.get("ranking_data", {}).get("ranking_string", None),
            "weekday_text": location_data.get("hours", {}).get("weekday_text", None)
        }
        data.append(location_info)

# Create DataFrame
df = pd.DataFrame(data)

df

,location_id,name,latitude,longitude,website,rating,ranking_string,weekday_text
0,8130064,New York City Subway,40.75588,-73.98698,https://new.mta.info/,4.0,"#186 of 2,165 things to do in New York City","[Monday: 00:00 - 23:59, Tuesday: 00:00 - 23:59..."
1,12423267,Secret Food Tours New York City,40.73123,-74.00169,http://www.secretfoodtours.com/new-york/,5.0,#1 of 336 Food &amp; Drink in New York City,"[Monday: 11:00 - 23:00, Tuesday: 11:00 - 23:00..."
2,4005106,Helicopter New York City,40.70191,-74.01015,http://www.helicopternewyorkcity.com,4.5,"#297 of 1,511 Tours in New York City","[Monday: 09:00 - 17:00, Tuesday: 09:00 - 17:00..."


In [2]:
import requests
url = "https://api.content.tripadvisor.com/api/v1/location/8130064/photos?language=en&key=150802BD11A24ADDBA3D4E7C0A8B2A67"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{
  "data": [
    {
      "id": 355479562,
      "is_blessed": false,
      "caption": "Passing by Citi Field in Queens",
      "published_date": "2018-10-29T06:05:30.013Z",
      "images": {
        "thumbnail": {
          "height": 50,
          "width": 50,
          "url": "https://media-cdn.tripadvisor.com/media/photo-t/15/30/30/0a/passing-by-citi-field.jpg"
        },
        "small": {
          "height": 150,
          "width": 150,
          "url": "https://media-cdn.tripadvisor.com/media/photo-l/15/30/30/0a/passing-by-citi-field.jpg"
        },
        "medium": {
          "height": 188,
          "width": 250,
          "url": "https://media-cdn.tripadvisor.com/media/photo-f/15/30/30/0a/passing-by-citi-field.jpg"
        },
        "large": {
          "height": 412,
          "width": 550,
          "url": "https://media-cdn.tripadvisor.com/media/photo-s/15/30/30/0a/passing-by-citi-field.jpg"
        },
        "original": {
          "height": 960,
          "width": 128

In [5]:
# Convert JSON response to Python dictionary
data = response.json()

# Extract names and URLs from the data
for photo in data['data']:
    print("Caption:", photo['caption'])
    print(" original URL:", photo['images']['original']['url'])
    print(" thumbnail URL:", photo['images']['thumbnail']['url'])
    print(" small URL:", photo['images']['small']['url'])
    print(" medium URL:", photo['images']['medium']['url'])
    print(" large URL:", photo['images']['large']['url'])

Caption: Passing by Citi Field in Queens
 original URL: https://media-cdn.tripadvisor.com/media/photo-m/1280/15/30/30/0a/passing-by-citi-field.jpg
 thumbnail URL: https://media-cdn.tripadvisor.com/media/photo-t/15/30/30/0a/passing-by-citi-field.jpg
 small URL: https://media-cdn.tripadvisor.com/media/photo-l/15/30/30/0a/passing-by-citi-field.jpg
 medium URL: https://media-cdn.tripadvisor.com/media/photo-f/15/30/30/0a/passing-by-citi-field.jpg
 large URL: https://media-cdn.tripadvisor.com/media/photo-s/15/30/30/0a/passing-by-citi-field.jpg
Caption: getlstd_property_photo
 original URL: https://media-cdn.tripadvisor.com/media/photo-o/07/dd/7f/68/getlstd-property-photo.jpg
 thumbnail URL: https://media-cdn.tripadvisor.com/media/photo-t/07/dd/7f/68/getlstd-property-photo.jpg
 small URL: https://media-cdn.tripadvisor.com/media/photo-l/07/dd/7f/68/getlstd-property-photo.jpg
 medium URL: https://media-cdn.tripadvisor.com/media/photo-f/07/dd/7f/68/getlstd-property-photo.jpg
 large URL: https://

# TicketMaster API - 5000 free calls a day - no card on account needed

In [27]:
API_KEY = 'XXX'
url = 'https://app.ticketmaster.com/discovery/v2/events.json'
params = {
    'apikey': API_KEY,
    'city': 'New York',
    'classificationName': 'music',  # Adjust for different types of events
    'size': 100  # Number of events to retrieve
}

response = requests.get(url, params=params)

data = response.json()

events = []

if '_embedded' in data and 'events' in data['_embedded']:
    for event in data['_embedded']['events']:
        events.append({
            'name': event['name'],
            'date': event['dates']['start']['localDate'],
            'time': event['dates']['start'].get('localTime', 'TBA'),
            'venue': event['_embedded']['venues'][0]['name'],
            'city': event['_embedded']['venues'][0]['city']['name'],
            'state': event['_embedded']['venues'][0]['state']['name']
        })
events

[{'name': 'The The - Ensouled World Tour',
  'date': '2024-10-17',
  'time': '20:30:00',
  'venue': 'Beacon Theatre',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-06-02',
  'time': '14:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-06-09',
  'time': '14:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-06-18',
  'time': '19:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-07-12',
  'time': '20:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New York',
  'state': 'New York'},
 {'name': 'Little Shop of Horrors',
  'date': '2024-07-24',
  'time': '14:00:00',
  'venue': 'Westside Theatre Upstairs - NY',
  'city': 'New Yo

In [18]:
API_KEY = 'XXX'
url = 'https://app.ticketmaster.com/discovery/v2/events.json'
params = {
    'apikey': API_KEY,
    'city': 'New York',
    'classificationName': 'music',  # Adjust for different types of events
    'size': 100  # Number of events to retrieve
}

response = requests.get(url, params=params)

data = response.json()

events = []

if '_embedded' in data and 'events' in data['_embedded']:
    for event in data['_embedded']['events']:
        events.append({
            'name': event['name'],
            'date': event['dates']['start']['localDate'],
            'time': event['dates']['start'].get('localTime', 'TBA'),
            'venue': event['_embedded']['venues'][0]['name'],
            'city': event['_embedded']['venues'][0]['city']['name'],
            'state': event['_embedded']['venues'][0]['state']['name']
        })

df_events = pd.DataFrame(events)
print(df_events)

                             name        date      time  \
0   The The - Ensouled World Tour  2024-10-17  20:30:00   
1          Little Shop of Horrors  2024-06-02  14:00:00   
2          Little Shop of Horrors  2024-06-09  14:00:00   
3          Little Shop of Horrors  2024-06-18  19:00:00   
4          Little Shop of Horrors  2024-07-12  20:00:00   
..                            ...         ...       ...   
95         Little Shop of Horrors  2024-08-28  19:00:00   
96         Little Shop of Horrors  2024-08-13  19:00:00   
97         Little Shop of Horrors  2024-08-24  14:00:00   
98         Little Shop of Horrors  2024-08-21  14:00:00   
99         Little Shop of Horrors  2024-08-14  14:00:00   

                             venue      city     state  
0                   Beacon Theatre  New York  New York  
1   Westside Theatre Upstairs - NY  New York  New York  
2   Westside Theatre Upstairs - NY  New York  New York  
3   Westside Theatre Upstairs - NY  New York  New York  
4   We

In [19]:
df_events

,name,date,time,venue,city,state
0,The The - Ensouled World Tour,2024-10-17,20:30:00,Beacon Theatre,New York,New York
1,Little Shop of Horrors,2024-06-02,14:00:00,Westside Theatre Upstairs - NY,New York,New York
2,Little Shop of Horrors,2024-06-09,14:00:00,Westside Theatre Upstairs - NY,New York,New York
3,Little Shop of Horrors,2024-06-18,19:00:00,Westside Theatre Upstairs - NY,New York,New York
4,Little Shop of Horrors,2024-07-12,20:00:00,Westside Theatre Upstairs - NY,New York,New York
...,...,...,...,...,...,...
95,Little Shop of Horrors,2024-08-28,19:00:00,Westside Theatre Upstairs - NY,New York,New York
96,Little Shop of Horrors,2024-08-13,19:00:00,Westside Theatre Upstairs - NY,New York,New York
97,Little Shop of Horrors,2024-08-24,14:00:00,Westside Theatre Upstairs - NY,New York,New York
98,Little Shop of Horrors,2024-08-21,14:00:00,Westside Theatre Upstairs - NY,New York,New York


In [20]:
API_KEY = 'XXX'
url = 'https://app.ticketmaster.com/discovery/v2/events.json'
params = {
    'apikey': API_KEY,
    'city': 'New York',
}

response = requests.get(url, params=params)
data = response.json()

events = []

if '_embedded' in data and 'events' in data['_embedded']:
    for event in data['_embedded']['events']:
        events.append({
            'name': event['name'],
            'date': event['dates']['start']['localDate'],
            'time': event['dates']['start'].get('localTime', 'TBA'),
            'venue': event['_embedded']['venues'][0]['name'],
            'city': event['_embedded']['venues'][0]['city']['name'],
            'state': event['_embedded']['venues'][0]['state']['name']
        })

df_events_1 = pd.DataFrame(events)
df_events_1

,name,date,time,venue,city,state
0,Giants 100: A Night With Legends,2024-06-20,18:30:00,The Theater at MSG,New York,New York
1,The The - Ensouled World Tour,2024-10-17,20:30:00,Beacon Theatre,New York,New York
2,Harry Potter and the Cursed Child,2024-05-31,19:00:00,Lyric Theatre - NY,New York,New York
3,Harry Potter and the Cursed Child,2024-06-14,19:00:00,Lyric Theatre - NY,New York,New York
4,Harry Potter and the Cursed Child,2024-06-01,13:00:00,Lyric Theatre - NY,New York,New York
5,Harry Potter and the Cursed Child,2024-06-29,19:00:00,Lyric Theatre - NY,New York,New York
6,Harry Potter and the Cursed Child,2024-06-08,13:00:00,Lyric Theatre - NY,New York,New York
7,Harry Potter and the Cursed Child,2024-07-26,19:00:00,Lyric Theatre - NY,New York,New York
8,Harry Potter and the Cursed Child,2024-07-28,13:00:00,Lyric Theatre - NY,New York,New York
9,Harry Potter and the Cursed Child,2024-08-15,19:00:00,Lyric Theatre - NY,New York,New York


In [24]:
df_events_1['date'] = pd.to_datetime(df_events_1['date'])

# Sort events by 'date' in ascending order
df_events_1 = df_events_1.sort_values(by='date').reset_index(drop=True)

(df_events_1)

,name,date,time,venue,city,state
0,Harry Potter and the Cursed Child,2024-05-31,19:00:00,Lyric Theatre - NY,New York,New York
1,Harry Potter and the Cursed Child,2024-06-01,13:00:00,Lyric Theatre - NY,New York,New York
2,Harry Potter and the Cursed Child,2024-06-04,19:00:00,Lyric Theatre - NY,New York,New York
3,Harry Potter and the Cursed Child,2024-06-08,13:00:00,Lyric Theatre - NY,New York,New York
4,Harry Potter and the Cursed Child,2024-06-14,19:00:00,Lyric Theatre - NY,New York,New York
5,Harry Potter and the Cursed Child,2024-06-18,19:00:00,Lyric Theatre - NY,New York,New York
6,Giants 100: A Night With Legends,2024-06-20,18:30:00,The Theater at MSG,New York,New York
7,Harry Potter and the Cursed Child,2024-06-22,19:00:00,Lyric Theatre - NY,New York,New York
8,Harry Potter and the Cursed Child,2024-06-26,19:00:00,Lyric Theatre - NY,New York,New York
9,Harry Potter and the Cursed Child,2024-06-26,13:00:00,Lyric Theatre - NY,New York,New York


## Ticketmaster attraction Data not much use

In [31]:
url = "https://app.ticketmaster.com/discovery/v2/attractions.json"
params = {
    'apikey': 'XXX'
}

response = requests.get(url, params=params)
json_data = response.json()
print(json_data)

{'_embedded': {'attractions': [{'name': 'Brett Young', 'type': 'attraction', 'id': 'K8vZ9178Ee7', 'test': False, 'url': 'https://www.ticketmaster.com/brett-young-tickets/artist/1810116', 'locale': 'en-us', 'externalLinks': {'twitter': [{'url': 'https://twitter.com/BrettYoungMusic'}], 'itunes': [{'url': 'https://itunes.apple.com/us/artist/id1084576956'}], 'facebook': [{'url': 'https://www.facebook.com/brettyoungmusicpage'}], 'wiki': [{'url': 'https://en.wikipedia.org/wiki/Brett_Young_(singer)'}], 'instagram': [{'url': 'https://www.instagram.com/brettyoungmusic/'}, {'url': 'https://www.instagram.com/brettyoungmusic'}], 'musicbrainz': [{'id': '2ec5396e-4fe9-4918-867f-34370e012093'}], 'homepage': [{'url': 'http://brettyoungmusic.com/'}]}, 'images': [{'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/250/0a67289f-8424-47a0-950a-ce8c403c3250_TABLET_LANDSCAPE_3_2.jpg', 'width': 1024, 'height': 683, 'fallback': False}, {'ratio': '4_3', 'url': 'https://s1.ticketm.net/dam/a/250/0a67289f-8424-

In [34]:
attractions = json_data['_embedded']['attractions']
    

for attraction in attractions:
    print(attraction['name'])

Brett Young
Pete Davidson
Club 90s
Ben Howard
Isabel LaRosa
James Kennedy
10 Years
Magdalena Bay
Omid Djalili
Mehro
Barry Can't Swim
Jon Dore
Mindchatter
Been Stellar
Gaz Coombes
GINMANBLACHMANDAHL
Emil De Waal
P!NK
Phoenix Suns
PARTYNEXTDOOR


In [35]:
for attraction in attractions:
        name = attraction['name']
        genre = attraction['classifications'][0]['genre']['name']
        sub_genre = attraction['classifications'][0]['subGenre']['name']
        
        print("Name:", name)
        print("Genre:", genre)
        print("Sub Genre:", sub_genre)
        print()

Name: Brett Young
Genre: Country
Sub Genre: Country

Name: Pete Davidson
Genre: Comedy
Sub Genre: Comedy

Name: Club 90s
Genre: Pop
Sub Genre: Pop

Name: Ben Howard
Genre: Rock
Sub Genre: Pop

Name: Isabel LaRosa
Genre: Pop
Sub Genre: Pop

Name: James Kennedy
Genre: Dance/Electronic
Sub Genre: Dance/Electronic

Name: 10 Years
Genre: Rock
Sub Genre: Alternative Rock

Name: Magdalena Bay
Genre: Pop
Sub Genre: Pop

Name: Omid Djalili
Genre: Comedy
Sub Genre: Comedy

Name: Mehro
Genre: Pop
Sub Genre: Pop

Name: Barry Can't Swim
Genre: Dance/Electronic
Sub Genre: Dance/Electronic

Name: Jon Dore
Genre: Comedy
Sub Genre: Comedy

Name: Mindchatter
Genre: Dance/Electronic
Sub Genre: Dance/Electronic

Name: Been Stellar
Genre: Rock
Sub Genre: Rock

Name: Gaz Coombes
Genre: Rock
Sub Genre: Pop

Name: GINMANBLACHMANDAHL
Genre: Jazz
Sub Genre: Jazz

Name: Emil De Waal
Genre: Jazz
Sub Genre: Jazz

Name: P!NK
Genre: Rock
Sub Genre: Pop

Name: Phoenix Suns
Genre: Basketball
Sub Genre: NBA

Name: PART

# Google Places API is decent but only returns up to 60 results (3 pages of 20 results) per search so coordiantes will have to be changed over and over to cover whole area and find all attractions

In [29]:
API_KEY = 'XXX'
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Define parameters for the request
params = {
    'key': API_KEY,
    'location': '40.7128,-74.0060',  # New York City coordinates
    'radius': 5000,  # Radius in meters MAX 5000
    'type': 'tourist_attraction',  # Type of place to search for
    'keyword': 'attraction'  # Keyword to search for (optional)
}

# Send GET request 
response = requests.get(base_url, params=params)
results = response.json()

# Extract relevant information from the response
attractions = []
if 'results' in results:
    for place in results['results']:
        attraction = {
            'name': place.get('name', 'N/A'),
            'address': place.get('vicinity', 'N/A'),
             'keyword': 'attraction',
            'rating': place.get('rating', 'N/A'),
            
            'maxResultCount': 100  # Maximum number of results to retrieve / only returned 20 anyway
        }
        attractions.append(attraction)

# Print the list of attractions
for attraction in attractions:
    print(attraction)

{'name': 'Door to Nowhere', 'address': '156 W 42nd St, New York', 'keyword': 'attraction', 'rating': 4.6, 'maxResultCount': 100}
{'name': 'Trinity Place Skybridge', 'address': '1-11 Thames St, New York', 'keyword': 'attraction', 'rating': 4.7, 'maxResultCount': 100}
{'name': 'The Channel Gardens', 'address': 'New York', 'keyword': 'attraction', 'rating': 4.6, 'maxResultCount': 100}
{'name': 'Washington Square Fountain', 'address': 'Washington Square N, New York', 'keyword': 'attraction', 'rating': 4.7, 'maxResultCount': 100}
{'name': 'Joey Ramone Place', 'address': '325 Bowery, New York', 'keyword': 'attraction', 'rating': 4.6, 'maxResultCount': 100}
{'name': 'Each zone, Every One, Equal All', 'address': '200 W 42nd St, New York', 'keyword': 'attraction', 'rating': 3.7, 'maxResultCount': 100}
{'name': 'New York City Elevated View Point', 'address': '80 Furman St, Brooklyn', 'keyword': 'attraction', 'rating': 4.8, 'maxResultCount': 100}
{'name': 'Questing NY', 'address': '9 Maiden Ln, N

In [40]:
# Define API key and base URL
API_KEY = 'XXX'
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Define parameters for the request
params = {
    'key': API_KEY,
    'location': '40.7128,-74.0060',  # New York City coordinates
    'radius': 5000,  # Radius in meters MAX 5000
    'type': 'tourist_attraction',  # Type of place to search for
    'keyword': 'attraction',  # Keyword to search for (optional)
    'maxResultCount': 100  # Maximum number of results to retrieve
}

# Send GET request
response = requests.get(base_url, params=params)
results = response.json()

# Function to get place details by place ID
def get_place_details(place_id):
    details_url = 'https://maps.googleapis.com/maps/api/place/details/json'
    details_params = {
        'key': API_KEY,
        'place_id': place_id,
        'fields': 'name,formatted_address,opening_hours,rating,geometry'
    }
    details_response = requests.get(details_url, params=details_params)
    details_result = details_response.json()
    return details_result['result']

# Extract relevant information from the response
attractions = []
if 'results' in results:
    for place in results['results']:
        place_details = get_place_details(place['place_id'])
        attraction = {
            'name': place_details.get('name', 'N/A'),
            'address': place_details.get('formatted_address', 'N/A'),
            'rating': place_details.get('rating', 'N/A'),
            'opening_hours': place_details.get('opening_hours', {}).get('weekday_text', 'N/A'),
            'latitude': place_details.get('geometry', {}).get('location', {}).get('lat', 'N/A'),
            'longitude': place_details.get('geometry', {}).get('location', {}).get('lng', 'N/A')
        }
        attractions.append(attraction)

# Print the list of attractions
for attraction in attractions:
    print(attraction)

{'name': 'Door to Nowhere', 'address': '156 W 42nd St, New York, NY 10036, USA', 'rating': 4.6, 'opening_hours': ['Monday: Open 24 hours', 'Tuesday: Open 24 hours', 'Wednesday: Open 24 hours', 'Thursday: Open 24 hours', 'Friday: Open 24 hours', 'Saturday: Open 24 hours', 'Sunday: Open 24 hours'], 'latitude': 40.75563889999999, 'longitude': -73.9863542}
{'name': 'Trinity Place Skybridge', 'address': '1-11 Thames St, New York, NY 10006, USA', 'rating': 4.7, 'opening_hours': ['Monday: Open 24 hours', 'Tuesday: Open 24 hours', 'Wednesday: Open 24 hours', 'Thursday: Open 24 hours', 'Friday: Open 24 hours', 'Saturday: Open 24 hours', 'Sunday: Open 24 hours'], 'latitude': 40.7090585, 'longitude': -74.01152789999999}
{'name': 'The Channel Gardens', 'address': 'New York, NY 10020, USA', 'rating': 4.6, 'opening_hours': ['Monday: Open 24 hours', 'Tuesday: Open 24 hours', 'Wednesday: Open 24 hours', 'Thursday: Open 24 hours', 'Friday: Open 24 hours', 'Saturday: Open 24 hours', 'Sunday: Open 24 hou

In [35]:
import requests
import pandas as pd

# Define API key and base URL
API_KEY = 'XXX'
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Define parameters for the request
params = {
    'key': API_KEY,
    'location': '40.7128,-74.0060',  # New York City coordinates
    'radius': 5000,  # Radius in meters (max 50000)
    'type': 'tourist_attraction',  # Type of place to search for
    'keyword': 'attraction',  # Keyword to search for (optional)
}

# Function to get place details by place ID
def get_place_details(place_id):
    details_url = 'https://maps.googleapis.com/maps/api/place/details/json'
    details_params = {
        'key': API_KEY,
        'place_id': place_id,
        'fields': 'address_component,adr_address,business_status,formatted_phone_number,geometry,icon,icon_mask_base_uri,international_phone_number,name,opening_hours,permanently_closed,photos,place_id,plus_code,price_level,rating,reviews,types,url,user_ratings_total,utc_offset,vicinity,website'
    }
    details_response = requests.get(details_url, params=details_params)
    details_result = details_response.json()
    return details_result['result']

# Send GET request to get initial list of places
response = requests.get(base_url, params=params)
results = response.json()

# Extract relevant information from the response
attractions = []
if 'results' in results:
    for place in results['results']:
        place_details = get_place_details(place['place_id'])
        attraction = {
            'address_component': place_details.get('address_components', 'N/A'),
            'adr_address': place_details.get('adr_address', 'N/A'),
            'business_status': place_details.get('business_status', 'N/A'),
            'formatted_phone_number': place_details.get('formatted_phone_number', 'N/A'),
            'geometry': place_details.get('geometry', 'N/A'),
            'icon': place_details.get('icon', 'N/A'),
            'icon_mask_base_uri': place_details.get('icon_mask_base_uri', 'N/A'),
            'international_phone_number': place_details.get('international_phone_number', 'N/A'),
            'name': place_details.get('name', 'N/A'),
            'opening_hours': place_details.get('opening_hours', {}).get('weekday_text', 'N/A'),
            'permanently_closed': place_details.get('permanently_closed', 'N/A'),
            'photos': place_details.get('photos', 'N/A'),
            'place_id': place_details.get('place_id', 'N/A'),
            'plus_code': place_details.get('plus_code', 'N/A'),
            'price_level': place_details.get('price_level', 'N/A'),
            'rating': place_details.get('rating', 'N/A'),
            'reviews': place_details.get('reviews', 'N/A'),
            'types': place_details.get('types', 'N/A'),
            'url': place_details.get('url', 'N/A'),
            'user_ratings_total': place_details.get('user_ratings_total', 'N/A'),
            'utc_offset': place_details.get('utc_offset', 'N/A'),
            'vicinity': place_details.get('vicinity', 'N/A'),
            'website': place_details.get('website', 'N/A')
        }
        attractions.append(attraction)

# Convert the list of attractions to a pandas DataFrame
df = pd.DataFrame(attractions)

# Display the DataFrame
(df)

,address_component,adr_address,business_status,formatted_phone_number,geometry,icon,icon_mask_base_uri,international_phone_number,name,opening_hours,...,plus_code,price_level,rating,reviews,types,url,user_ratings_total,utc_offset,vicinity,website
0,"[{'long_name': 'New York', 'short_name': 'New ...","<span class=""locality"">New York</span>, <span ...",OPERATIONAL,N/A,"{'location': {'lat': 40.7585013, 'lng': -73.97...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,N/A,The Channel Gardens,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",...,"{'compound_code': 'Q25C+CQ New York, NY, USA',...",N/A,4.6,"[{'author_name': 'Sean Ellis', 'author_url': '...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=2026719009605146857,511,-240,New York,https://www.rockefellercenter.com/attractions/...
1,"[{'long_name': 'Washington Square North', 'sho...","<span class=""street-address"">Washington Square...",OPERATIONAL,(212) 639-9675,"{'location': {'lat': 40.7308514, 'lng': -73.99...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 212-639-9675,Washington Square Fountain,"[Monday: 6:00 AM – 1:00 AM, Tuesday: 6:00 AM –...",...,"{'compound_code': 'P2J3+82 New York, NY, USA',...",N/A,4.7,"[{'author_name': 'Elyssa Ymb', 'author_url': '...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=13332122140886915890,463,-240,"Washington Square North, New York",https://www.nycgovparks.org/parks/washington-s...
2,"[{'long_name': '156', 'short_name': '156', 'ty...","<span class=""street-address"">156 W 42nd St</sp...",OPERATIONAL,N/A,"{'location': {'lat': 40.75563889999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,N/A,Door to Nowhere,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",...,"{'compound_code': 'Q247+7F New York, NY, USA',...",N/A,4.6,"[{'author_name': 'Alex Weber', 'author_url': '...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=14747897764269397466,9,-240,"156 West 42nd Street, New York",N/A
3,"[{'long_name': '1-11', 'short_name': '1-11', '...","<span class=""street-address"">1-11 Thames St</s...",OPERATIONAL,N/A,"{'location': {'lat': 40.7090585, 'lng': -74.01...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,N/A,Trinity Place Skybridge,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",...,"{'compound_code': 'PX5Q+J9 New York, NY, USA',...",N/A,4.7,"[{'author_name': 'George Messinia', 'author_ur...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=7944431488392360646,3,-240,"1-11 Thames Street, New York",N/A
4,"[{'long_name': 'Pierrepont Place', 'short_name...","Montague St &amp;, <span class=""street-address...",OPERATIONAL,(212) 639-9675,"{'location': {'lat': 40.69641729999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 212-639-9675,Brooklyn Heights Promenade,"[Monday: 6:00 AM – 1:00 AM, Tuesday: 6:00 AM –...",...,"{'compound_code': 'M2W2+HW Brooklyn, NY, USA',...",N/A,4.8,"[{'author_name': 'Marcos Perez', 'author_url':...","[park, tourist_attraction, point_of_interest, ...",https://maps.google.com/?cid=13697143868944734919,5943,-240,"Montague St &, Pierrepont Place, Brooklyn",http://www.nycgovparks.org/about/history/histo...
5,"[{'long_name': '200', 'short_name': '200', 'ty...","<span class=""street-address"">200 W 42nd St</sp...",OPERATIONAL,N/A,"{'location': {'lat': 40.755828, 'lng': -73.986...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,N/A,"Each zone, Every One, Equal All","[Monday: Open 24 hours, Tuesday: Open 24 hours...",...,"{'compound_code': 'Q247+88 New York, NY, USA',...",N/A,3.7,"[{'author_name': 'gustavo stecher', 'author_ur...","[tourist_a

In [46]:
for place in results['results']:
        place_details = get_place_details(place['place_id'])
        print(place['name'])
        print(place_details['icon_mask_base_uri'])

The Channel Gardens
https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet
Washington Square Fountain
https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet
Door to Nowhere
https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet
Trinity Place Skybridge
https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet
Brooklyn Heights Promenade
https://maps.gstatic.com/mapfiles/place_api/icons/v2/tree_pinlet
Each zone, Every One, Equal All
https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet
Questing NY
https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet
Morris Canal Park
https://maps.gstatic.com/mapfiles/place_api/icons/v2/tree_pinlet
New York City Elevated View Point
https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet
Flag Plaza
https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet
Grand Ferry Park
https://maps.gstatic.com/mapfiles/place_api/icons/v2/tree_pinlet
SeaGlass Carousel
https

In [48]:
import requests
import pandas as pd

# Define API key
API_KEY = 'XXX'

# Define the base URL for the nearby search
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Define parameters for the request
params = {
    'key': API_KEY,
    'location': '40.7128,-74.0060',  # New York City coordinates
    'radius': 5000,  # Radius in meters (max 50000)
    'type': 'tourist_attraction',  # Type of place to search for
    'keyword': 'attraction',  # Keyword to search for (optional)
}

# Function to get place details by place ID
def get_place_details(place_id):
    details_url = 'https://maps.googleapis.com/maps/api/place/details/json'
    details_params = {
        'key': API_KEY,
        'place_id': place_id,
        'fields': 'address_component,adr_address,business_status,formatted_phone_number,geometry,icon,icon_mask_base_uri,international_phone_number,name,opening_hours,permanently_closed,photos,place_id,plus_code,price_level,rating,reviews,types,url,user_ratings_total,utc_offset,vicinity,website'
    }
    details_response = requests.get(details_url, params=details_params)
    details_result = details_response.json()
    return details_result['result']

# Function to get photo URL by photo reference
def get_photo_url(photo_reference):
    photo_url = f'https://maps.googleapis.com/maps/api/place/photo?maxwidth=400&photo_reference={photo_reference}&key={API_KEY}'
    return photo_url

# Send GET request to get initial list of places
response = requests.get(base_url, params=params)
results = response.json()

# Extract relevant information from the response
attractions = []
if 'results' in results:
    for place in results['results']:
        place_details = get_place_details(place['place_id'])
        photo_references = place_details.get('photos', [])
        photo_urls = [get_photo_url(photo['photo_reference']) for photo in photo_references]

        attraction = {
            'address_component': place_details.get('address_components', 'N/A'),
            'adr_address': place_details.get('adr_address', 'N/A'),
            'business_status': place_details.get('business_status', 'N/A'),
            'formatted_phone_number': place_details.get('formatted_phone_number', 'N/A'),
            'geometry': place_details.get('geometry', 'N/A'),
            'icon': place_details.get('icon', 'N/A'),
            'icon_mask_base_uri': place_details.get('icon_mask_base_uri', 'N/A'),
            'international_phone_number': place_details.get('international_phone_number', 'N/A'),
            'name': place_details.get('name', 'N/A'),
            'opening_hours': place_details.get('opening_hours', {}).get('weekday_text', 'N/A'),
            'permanently_closed': place_details.get('permanently_closed', 'N/A'),
            'photos': photo_urls if photo_urls else 'N/A',
            'place_id': place_details.get('place_id', 'N/A'),
            'plus_code': place_details.get('plus_code', 'N/A'),
            'price_level': place_details.get('price_level', 'N/A'),
            'rating': place_details.get('rating', 'N/A'),
            'reviews': place_details.get('reviews', 'N/A'),
            'types': place_details.get('types', 'N/A'),
            'url': place_details.get('url', 'N/A'),
            'user_ratings_total': place_details.get('user_ratings_total', 'N/A'),
            'utc_offset': place_details.get('utc_offset', 'N/A'),
            'vicinity': place_details.get('vicinity', 'N/A'),
            'website': place_details.get('website', 'N/A')
        }
        attractions.append(attraction)

# Convert the list of attractions to a pandas DataFrame
df = pd.DataFrame(attractions)

# Display the DataFrame
df

,address_component,adr_address,business_status,formatted_phone_number,geometry,icon,icon_mask_base_uri,international_phone_number,name,opening_hours,...,plus_code,price_level,rating,reviews,types,url,user_ratings_total,utc_offset,vicinity,website
0,"[{'long_name': '156', 'short_name': '156', 'ty...","<span class=""street-address"">156 W 42nd St</sp...",OPERATIONAL,N/A,"{'location': {'lat': 40.75563889999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,N/A,Door to Nowhere,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",...,"{'compound_code': 'Q247+7F New York, NY, USA',...",N/A,4.6,"[{'author_name': 'Alex Weber', 'author_url': '...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=14747897764269397466,9,-240,"156 West 42nd Street, New York",N/A
1,"[{'long_name': '1-11', 'short_name': '1-11', '...","<span class=""street-address"">1-11 Thames St</s...",OPERATIONAL,N/A,"{'location': {'lat': 40.7090585, 'lng': -74.01...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,N/A,Trinity Place Skybridge,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",...,"{'compound_code': 'PX5Q+J9 New York, NY, USA',...",N/A,4.7,"[{'author_name': 'George Messinia', 'author_ur...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=7944431488392360646,3,-240,"1-11 Thames Street, New York",N/A
2,"[{'long_name': 'New York', 'short_name': 'New ...","<span class=""locality"">New York</span>, <span ...",OPERATIONAL,N/A,"{'location': {'lat': 40.7585013, 'lng': -73.97...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,N/A,The Channel Gardens,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",...,"{'compound_code': 'Q25C+CQ New York, NY, USA',...",N/A,4.6,"[{'author_name': 'Sean Ellis', 'author_url': '...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=2026719009605146857,511,-240,New York,https://www.rockefellercenter.com/attractions/...
3,"[{'long_name': 'Washington Square North', 'sho...","<span class=""street-address"">Washington Square...",OPERATIONAL,(212) 639-9675,"{'location': {'lat': 40.7308514, 'lng': -73.99...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 212-639-9675,Washington Square Fountain,"[Monday: 6:00 AM – 1:00 AM, Tuesday: 6:00 AM –...",...,"{'compound_code': 'P2J3+82 New York, NY, USA',...",N/A,4.7,"[{'author_name': 'Elyssa Ymb', 'author_url': '...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=13332122140886915890,463,-240,"Washington Square North, New York",https://www.nycgovparks.org/parks/washington-s...
4,"[{'long_name': '325', 'short_name': '325', 'ty...","<span class=""street-address"">325 Bowery</span>...",OPERATIONAL,N/A,"{'location': {'lat': 40.7257543, 'lng': -73.99...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,N/A,Joey Ramone Place,N/A,...,"{'compound_code': 'P2G5+87 New York, NY, USA',...",N/A,4.6,"[{'author_name': 'NY_78Hz', 'author_url': 'htt...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=2947611350074760197,110,-240,"325 Bowery, New York",N/A
5,"[{'long_name': '200', 'short_name': '200', 'ty...","<span class=""street-address"">200 W 42nd St</sp...",OPERATIONAL,N/A,"{'location': {'lat': 40.755828, 'lng': -73.986...",https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,N/A,"Each zone, Every One, Equal All","[Monday: Open 24 hours, Tuesday: Open 24 hours...",...,"{'compound_code': 'Q247+88 New York, NY, USA',...",N/A,3.7,"[{'author_name': 'gustavo stecher', 'author_ur...","[tourist_attraction, point_of_interest, establ...",https://maps.google.com/?cid=13420458991264250737,3,-240,"200 West 42nd Street, New York",N/A
6,"[{'long_na

In [48]:
attractions_df = pd.DataFrame(attractions)
attractions_df

,name,address,rating,opening_hours,latitude,longitude
0,Door to Nowhere,"156 W 42nd St, New York, NY 10036, USA",4.6,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.755639,-73.986354
1,Trinity Place Skybridge,"1-11 Thames St, New York, NY 10006, USA",4.7,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.709058,-74.011528
2,The Channel Gardens,"New York, NY 10020, USA",4.6,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.758501,-73.978087
3,Joey Ramone Place,"325 Bowery, New York, NY 10003, USA",4.6,N/A,40.725754,-73.991806
4,Washington Square Fountain,"Washington Square N, New York, NY 10012, USA",4.7,"[Monday: 6:00 AM – 1:00 AM, Tuesday: 6:00 AM –...",40.730851,-73.997431
5,"Each zone, Every One, Equal All","200 W 42nd St, New York, NY 10036, USA",3.7,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.755828,-73.986639
6,New York City Elevated View Point,"80 Furman St, Brooklyn, NY 11201, USA",4.8,N/A,40.700937,-73.996036
7,Questing NY,"9 Maiden Ln, New York, NY 10038, USA",5.0,"[Monday: 8:00 AM – 8:00 PM, Tuesday: 8:00 AM –...",40.709625,-74.009337
8,Manhattan Bridge Pedestrian Path,"Brooklyn Heights, NY 11201, USA",4.6,N/A,40.708806,-73.991655
9,Silverstein Family Park,"Greenwich St, New York, NY 10007, USA",4.5,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",40.713194,-74.011497


# Yelp API testing

In [9]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Set your Yelp Fusion API key here
api_key = 'XXX'

headers = {'Authorization': f'Bearer {api_key}'}

# parameters 
params = {
    'location': 'New York, NY',
    'limit': 150,  # max number of events per request 
    'start_date': int((datetime.now() + timedelta(days=1)).timestamp())  # start date for future events
}

# Yelp Fusion API endpoint for events search
url = 'https://api.yelp.com/v3/events'

#API call
response = requests.get(url, headers=headers, params=params)
data = response.json()

# Check if response contains events
if 'events' in data:
    events = data['events']
    # Extract event details
    event_list = []
    for event in events:
        event_info = {
            'name': event.get('name'),
            'address': event.get('location', {}).get('address1'),
            'city': event.get('location', {}).get('city'),
            'state': event.get('location', {}).get('state'),
            'zip_code': event.get('location', {}).get('zip_code'),
            'latitude': event.get('latitude'),
            'longitude': event.get('longitude'),
            'category': event.get('category'),
            'description': event.get('description'),
            'event_site_url': event.get('event_site_url'),
            'image_url': event.get('image_url'),
            'is_free': event.get('is_free'),
            'time_start': event.get('time_start'),
            'time_end': event.get('time_end'),
            'attending_count': event.get('attending_count'),
            'interested_count': event.get('interested_count'),
            'is_canceled': event.get('is_canceled'),
            'is_official': event.get('is_official')
        }
        event_list.append(event_info)

    # Create DataFrame
    df = pd.DataFrame(event_list)

    # Display DataFrame
    print(df.head())

    # Save to CSV
    df.to_csv('yelp_future_events_nyc.csv', index=False)
else:
    print("No events found")

                                                name          address  \
0                              Museum Scavenger Hunt   281 Park Ave S   
1  2024 International Conference on Computational...     80 Spring St   
2  2024 International Conference on Intelligent V...     108 E 4th St   
3                                              RISK!  21 A Clinton St   
4                                  panty raid comedy  21 A Clinton St   

       city state zip_code   latitude  longitude         category  \
0  New York    NY    10010  40.739221 -73.986722      kids-family   
1  New York    NY    10012  40.722681 -73.998230            other   
2  New York    NY    10003  40.725534 -73.988352            other   
3  New York    NY    10002  40.720741 -73.984434            other   
4  New York    NY    10002  40.720741 -73.984434  performing-arts   

                                         description  \
0  Join us for a paw-some adventure at Fotografis...   
1  2024 International Conference o

In [10]:
df

,name,address,city,state,zip_code,latitude,longitude,category,description,event_site_url,image_url,is_free,time_start,time_end,attending_count,interested_count,is_canceled,is_official
0,Museum Scavenger Hunt,281 Park Ave S,New York,NY,10010,40.739221,-73.986722,kids-family,Join us for a paw-some adventure at Fotografis...,https://www.yelp.com/events/new-york-museum-sc...,https://s3-media4.fl.yelpcdn.com/ephoto/q7rjH4...,False,2024-09-15T14:00:00-04:00,None,1,0,False,False
1,2024 International Conference on Computational...,80 Spring St,New York,NY,10012,40.722681,-73.998230,other,2024 International Conference on Computational...,https://www.yelp.com/events/new-york-2024-inte...,https://s3-media3.fl.yelpcdn.com/ephoto/SVj3wU...,False,2024-07-19T09:00:00-04:00,2024-07-21T18:00:00-04:00,1,0,False,False
2,2024 International Conference on Intelligent V...,108 E 4th St,New York,NY,10003,40.725534,-73.988352,other,2024 International Conference on Intelligent V...,https://www.yelp.com/events/new-york-2024-inte...,https://s3-media4.fl.yelpcdn.com/ephoto/JDLO4u...,True,2024-09-27T09:00:00-04:00,2024-09-29T18:30:00-04:00,1,0,False,False
3,RISK!,21 A Clinton St,New York,NY,10002,40.720741,-73.984434,other,RISK! is the show where people tell true stori...,https://www.yelp.com/events/new-york-risk-5?ad...,https://s3-media4.fl.yelpcdn.com/ephoto/GdIWRa...,False,2024-07-18T21:30:00-04:00,None,1,0,False,False
4,panty raid comedy,21 A Clinton St,New York,NY,10002,40.720741,-73.984434,performing-arts,We're hosting the FIRST Panty Raid show of 202...,https://www.yelp.com/events/new-york-panty-rai...,https://s3-media3.fl.yelpcdn.com/ephoto/Rgfjfz...,False,2024-06-21T19:00:00-04:00,2024-06-21T21:00:00-04:00,1,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Voguing with Father DayDay of the House of Juicy,145 Bowery,New York,NY,10002,40.718957,-73.994412,performing-arts,Join us for a night of VOGUE with Father DayDa...,https://www.yelp.com/events/new-york-voguing-w...,https://s3-media1.fl.yelpcdn.com/ephoto/nMSK-t...,False,2024-06-21T17:00:00-04:00,2024-06-21T20:00:00-04:00,1,0,False,False
123,Shag with Pride: Trans Pleasure Workshop and S...,353 Bedford Ave,Brooklyn,NY,11211,40.711690,-73.962877,nightlife,"Come for the workshop, stay for the social. En...",https://www.yelp.com/events/brooklyn-shag-with...,https://s3-media3.fl.yelpcdn.com/ephoto/r0LPd-...,False,2024-06-06T19:00:00-04:00,2024-06-06T21:00:00-04:00,1,0,False,False
124,Friend of Flame Speed Dating; Queer Edition,353 Bedford Ave,Brooklyn,NY,11211,40.711690,-73.962877,nightlife,Friend or Flame is partnering with Moxy Brookl...,https://www.yelp.com/events/brooklyn-friend-of...,https://s3-media2.fl.yelpcdn.com/ephoto/4JTfv1...,False,2024-06-18T19:00:00-04:00,2024-06-18T22:00:00-04:00,1,0,False,False
125,Full Moon Rooftop Yoga and Astrology,353 Bedford Ave,Brooklyn,NY,11211,40.711690,-73.962877,sports-active-life,Join us on the rooftop with yoga and sound bow...,https://www.yelp.com/events/brooklyn-full-moon...,https://s3-media4.fl.yelpcdn.com/ephoto/Fb7wDH...,False,2024-06-21T10:00:00-04:00,2024-06-21T12:30:00-04:00,1,0,False,False


In [14]:
df['attending_count'].unique().tolist()

[1, 223, 2]

In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
latitude,127.0,40.720360,0.019820,40.653218,40.707206,40.721601,40.737259,40.749636
longitude,127.0,-73.990685,0.016538,-74.047224,-74.003412,-73.987834,-73.984497,-73.954938
attending_count,127.0,2.755906,19.698810,1.000000,1.000000,1.000000,1.000000,223.000000
interested_count,127.0,5.267717,57.944753,0.000000,0.000000,0.000000,0.000000,653.000000


In [19]:
filtered_df = df[df['attending_count'] == 223]
filtered_df

,name,address,city,state,zip_code,latitude,longitude,category,description,event_site_url,image_url,is_free,time_start,time_end,attending_count,interested_count,is_canceled,is_official
54,Celebrate APIHM in Manhattan,,New York,NY,,40.713,-74.0072,food-and-drink,"Taiyaki, rice rolls, and dragon tea? We're cel...",https://www.yelp.com/events/new-york-celebrate...,https://s3-media4.fl.yelpcdn.com/ephoto/OHdbih...,True,2024-05-20T00:00:00-04:00,2024-06-19T00:00:00-04:00,223,13,False,True


In [20]:
print(df.head())

                                                name          address  \
0                              Museum Scavenger Hunt   281 Park Ave S   
1  2024 International Conference on Computational...     80 Spring St   
2  2024 International Conference on Intelligent V...     108 E 4th St   
3                                              RISK!  21 A Clinton St   
4                                  panty raid comedy  21 A Clinton St   

       city state zip_code   latitude  longitude         category  \
0  New York    NY    10010  40.739221 -73.986722      kids-family   
1  New York    NY    10012  40.722681 -73.998230            other   
2  New York    NY    10003  40.725534 -73.988352            other   
3  New York    NY    10002  40.720741 -73.984434            other   
4  New York    NY    10002  40.720741 -73.984434  performing-arts   

                                         description  \
0  Join us for a paw-some adventure at Fotografis...   
1  2024 International Conference o

In [21]:
df['image_url']

0      https://s3-media4.fl.yelpcdn.com/ephoto/q7rjH4...
1      https://s3-media3.fl.yelpcdn.com/ephoto/SVj3wU...
2      https://s3-media4.fl.yelpcdn.com/ephoto/JDLO4u...
3      https://s3-media4.fl.yelpcdn.com/ephoto/GdIWRa...
4      https://s3-media3.fl.yelpcdn.com/ephoto/Rgfjfz...
                             ...                        
122    https://s3-media1.fl.yelpcdn.com/ephoto/nMSK-t...
123    https://s3-media3.fl.yelpcdn.com/ephoto/r0LPd-...
124    https://s3-media2.fl.yelpcdn.com/ephoto/4JTfv1...
125    https://s3-media4.fl.yelpcdn.com/ephoto/Fb7wDH...
126    https://s3-media2.fl.yelpcdn.com/ephoto/gDDjou...
Name: image_url, Length: 127, dtype: object

In [28]:
for event in events:
    image_url = event.get('image_url', 'N/A')
    website_url = event.get('event_site_url', 'N/A')
    print(event['name'])
    print(image_url)
    print(website_url)

Museum Scavenger Hunt
https://s3-media4.fl.yelpcdn.com/ephoto/q7rjH42CwozqD1x_1dT6xw/o.jpg
https://www.yelp.com/events/new-york-museum-scavenger-hunt?adjust_creative=vaXd63wEr8RrzPKFIuH35Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=vaXd63wEr8RrzPKFIuH35Q
2024 International Conference on Computational Linguistics and Natural Language Processing (CLNLP 2024)
https://s3-media3.fl.yelpcdn.com/ephoto/SVj3wU5RajhhNNZM6dqIhA/o.jpg
https://www.yelp.com/events/new-york-2024-international-conference-on-computational-linguistics-and-natural-language-processing-clnlp-2024?adjust_creative=vaXd63wEr8RrzPKFIuH35Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=vaXd63wEr8RrzPKFIuH35Q
2024 International Conference on Intelligent Vehicles (ICoIV 2024)
https://s3-media4.fl.yelpcdn.com/ephoto/JDLO4uTKShEvfjCCzu1qMw/o.jpg
https://www.yelp.com/events/new-york-2024-international-conference-on-intelligent-vehicles-icoiv-2024?adjust_creative=vaXd63wEr8RrzPKFIuH35Q&utm

In [18]:
import requests
import pandas as pd

# Define the Overpass API endpoint
overpass_url = "https://overpass-api.de/api/interpreter"

# Define the query to retrieve places of interest in Manhattan
query = """
[out:json];
(
  node["tourism"](40.6997,-74.0183,40.8774,-73.9105);
  way["tourism"](40.6997,-74.0183,40.8774,-73.9105);
  relation["tourism"](40.6997,-74.0183,40.8774,-73.9105);
);
out body;
>;
out skel qt;
"""

# Send the query to the Overpass API
response = requests.post(overpass_url, data=query)

# Parse the JSON response
data = response.json()
print(data)

{'version': 0.6, 'generator': 'Overpass API 0.7.62.1 084b4234', 'osm3s': {'timestamp_osm_base': '2024-06-06T15:27:14Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'node', 'id': 158801311, 'lat': 40.7251022, 'lon': -73.9795833, 'tags': {'alt_name': 'Loisaida', 'ele': '4', 'gnis:feature_id': '2062665', 'name': 'Alphabet City', 'name:ru': 'Алфабет-сити', 'name:sr': 'Алфабет сити', 'name:sr-Latn': 'Alfabet siti', 'place': 'neighbourhood', 'tourism': 'yes', 'wikidata': 'Q1156938'}}, {'type': 'node', 'id': 158809984, 'lat': 40.72288, 'lon': -73.9987505, 'tags': {'ele': '9', 'gnis:feature_id': '2045148', 'name': 'SoHo', 'name:ru': 'Сохо', 'name:sr': 'Сохо', 'name:sr-Latn': 'Soho', 'name:zh': '苏活区', 'name:zh-Hant': '蘇活區', 'place': 'neighbourhood', 'tourism': 'attraction', 'wikidata': 'Q461572', 'wikipedia': 'en:SoHo, Manhattan'}}, {'type': 'node', 'id': 158862484, 'lat': 40.7486231, 'lon': -73.

In [17]:
import pandas as pd
import requests

# Define the Overpass API endpoint
overpass_url = "https://overpass-api.de/api/interpreter"

# Define the query to retrieve places of interest in Manhattan
query = """
[out:json];
(
  node["tourism"](40.6997,-74.0183,40.8774,-73.9105);
  way["tourism"](40.6997,-74.0183,40.8774,-73.9105);
  relation["tourism"](40.6997,-74.0183,40.8774,-73.9105);
);
out body;
>;
out skel qt;
"""

# Send the query to the Overpass API
response = requests.post(overpass_url, data=query)

# Parse the JSON response
data = response.json()['elements']

# Extract all information for each node
nodes_info = []
for node in data:
    node_info = {
        'id': node['id'],
        'type': node['type'],
        'latitude': node.get('lat'),
        'longitude': node.get('lon'),
    }
    tags = node.get('tags', {})
    for key, value in tags.items():
        node_info[key] = value
    nodes_info.append(node_info)

# Create a DataFrame from the extracted data
df = pd.DataFrame(nodes_info)

# Display the DataFrame
df

,id,type,latitude,longitude,alt_name,ele,gnis:feature_id,name,name:ru,name:sr,...,admin_level,name:bg,name:ml,seamark:platform:category,seamark:type,closed,tunnel,addr:place,bicycle,old_railway_operator
0,158801311,node,40.725102,-73.979583,Loisaida,4,2062665,Alphabet City,Алфабет-сити,Алфабет сити,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,158809984,node,40.722880,-73.998750,NaN,9,2045148,SoHo,Сохо,Сохо,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,158862484,node,40.748623,-73.971389,NaN,18,2083304,Tudor City,Тьюдор-сити,Тудор сити,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,320948228,node,40.705751,-74.002906,NaN,NaN,NaN,South Street Seaport,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,357618558,node,40.732267,-73.997255,NaN,7,2080738,"New York Studio School of Drawing, Painting an...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,104917339,way,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9990,104917349,way,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9991,104917358,way,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9992,285900676,way,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df[(df['tourism'] == 'yes') | (df['tourism'] == 'attraction')]

,id,type,latitude,longitude,alt_name,ele,gnis:feature_id,name,name:ru,name:sr,...,admin_level,name:bg,name:ml,seamark:platform:category,seamark:type,closed,tunnel,addr:place,bicycle,old_railway_operator
0,158801311,node,40.725102,-73.979583,Loisaida,4,2062665,Alphabet City,Алфабет-сити,Алфабет сити,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,158809984,node,40.722880,-73.998750,NaN,9,2045148,SoHo,Сохо,Сохо,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,158862484,node,40.748623,-73.971389,NaN,18,2083304,Tudor City,Тьюдор-сити,Тудор сити,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,320948228,node,40.705751,-74.002906,NaN,NaN,NaN,South Street Seaport,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,357618558,node,40.732267,-73.997255,NaN,7,2080738,"New York Studio School of Drawing, Painting an...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,2154552,building,NaN,NaN,NaN,NaN,NaN,Saint Patrick’s Cathedral,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2036,3624833,multipolygon,NaN,NaN,Cathedral Church of Saint John the Divine in t...,37,946040,Cathedral of Saint John the Divine,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2038,6424088,multipolygon,NaN,NaN,NaN,54,967218,The Cloisters,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fort Tryon Park,NaN,NaN
2039,7141751,multipolygon,NaN,NaN,NaN,NaN,NaN,The High Line,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NYC


In [29]:
print(df.isnull().sum())

id                         0
type                       0
latitude                 576
longitude                576
alt_name                9926
                        ... 
closed                  9993
tunnel                  9993
addr:place              9993
bicycle                 9993
old_railway_operator    9993
Length: 335, dtype: int64


In [36]:
import requests
import pandas as pd

# Define the Overpass API endpoint
overpass_url = "https://overpass-api.de/api/interpreter"

# Define the query to retrieve places of interest in Manhattan
query = """
[out:json];
(
  node["tourism"](40.6997,-74.0183,40.8774,-73.9105);
  way["tourism"](40.6997,-74.0183,40.8774,-73.9105);
  relation["tourism"](40.6997,-74.0183,40.8774,-73.9105);
);
out body;
>;
out skel qt;
"""

# Send the query to the Overpass API
response = requests.post(overpass_url, data=query)

# Parse the JSON response
data = response.json()

# Extract relevant information
places = []
for element in data['elements']:
    if 'tags' in element:
        place = {
            'id': element.get('id'),
            'name': element['tags'].get('name'),
            'lat': element.get('lat'),
            'lon': element.get('lon'),
            'tourism': element['tags'].get('tourism'),
            'addr:city': element['tags'].get('addr:city'),
            'addr:postcode': element['tags'].get('addr:postcode'),
            'addr:state': element['tags'].get('addr:state'),
            'addr:street': element['tags'].get('addr:street'),
            'website': element['tags'].get('website'),
            'opening_hours': element['tags'].get('opening_hours'),
            'phone': element['tags'].get('phone'),
            'email': element['tags'].get('email'),
            'wheelchair': element['tags'].get('wheelchair'),
            'ele': element['tags'].get('ele'),
            'source': element['tags'].get('source'),
            'wikidata': element['tags'].get('wikidata'),
            'wikipedia': element['tags'].get('wikipedia'),
        }
        places.append(place)

# Create DataFrame
df = pd.DataFrame(places)

# Display DataFrame
df

,id,name,lat,lon,tourism,addr:city,addr:postcode,addr:state,addr:street,website,opening_hours,phone,email,wheelchair,ele,source,wikidata,wikipedia
0,158801311,Alphabet City,40.725102,-73.979583,yes,None,None,None,None,None,None,None,None,None,4,None,Q1156938,None
1,158809984,SoHo,40.722880,-73.998750,attraction,None,None,None,None,None,None,None,None,None,9,None,Q461572,"en:SoHo, Manhattan"
2,158862484,Tudor City,40.748623,-73.971389,yes,None,None,None,None,None,None,None,None,None,18,None,Q549206,None
3,320948228,South Street Seaport,40.705751,-74.002906,attraction,None,None,None,None,None,None,None,None,yes,None,None,Q223000,en:South Street Seaport
4,357618558,"New York Studio School of Drawing, Painting an...",40.732267,-73.997255,yes,None,None,None,None,None,None,None,None,None,7,None,Q7014742,"en:New York Studio School of Drawing, Painting..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,7281773,Museum of Jewish Heritage,NaN,NaN,museum,None,None,None,None,https://mjhnyc.org/,None,None,None,None,None,None,Q839985,en:Museum of Jewish Heritage
2041,9771573,The Plaza,NaN,NaN,hotel,None,10019,None,5th Avenue,https://www.theplazany.com,None,+1 212-759-3000,None,None,None,None,Q1066676,en:Plaza Hotel
2042,11510854,Serpentine Structures,NaN,NaN,artwork,None,None,None,None,https://hudsonriverpark.org/activities/serpent...,None,None,None,None,None,None,None,None
2043,16231018,Vessel,NaN,NaN,artwork,None,None,None,None,https://www.hudsonyardsnewyork.com/discover/ve...,None,None,None,yes,None,None,Q27988215,en:Vessel (structure)


In [34]:
print(df.isnull().sum())

id                  0
name              611
lat               517
lon               517
tourism             0
addr:city        1425
addr:postcode    1322
addr:state       1498
addr:street      1226
website          1215
opening_hours    1795
phone            1519
email            1810
wheelchair       1783
ele              1975
source           1986
wikidata         1694
wikipedia        1783
dtype: int64


In [35]:
df

,id,name,lat,lon,tourism,addr:city,addr:postcode,addr:state,addr:street,website,opening_hours,phone,email,wheelchair,ele,source,wikidata,wikipedia
0,158801311,Alphabet City,40.725102,-73.979583,yes,None,None,None,None,None,None,None,None,None,4,None,Q1156938,None
1,158809984,SoHo,40.722880,-73.998750,attraction,None,None,None,None,None,None,None,None,None,9,None,Q461572,"en:SoHo, Manhattan"
2,158862484,Tudor City,40.748623,-73.971389,yes,None,None,None,None,None,None,None,None,None,18,None,Q549206,None
3,320948228,South Street Seaport,40.705751,-74.002906,attraction,None,None,None,None,None,None,None,None,yes,None,None,Q223000,en:South Street Seaport
4,357618558,"New York Studio School of Drawing, Painting an...",40.732267,-73.997255,yes,None,None,None,None,None,None,None,None,None,7,None,Q7014742,"en:New York Studio School of Drawing, Painting..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2040,7281773,Museum of Jewish Heritage,NaN,NaN,museum,None,None,None,None,https://mjhnyc.org/,None,None,None,None,None,None,Q839985,en:Museum of Jewish Heritage
2041,9771573,The Plaza,NaN,NaN,hotel,None,10019,None,5th Avenue,https://www.theplazany.com,None,+1 212-759-3000,None,None,None,None,Q1066676,en:Plaza Hotel
2042,11510854,Serpentine Structures,NaN,NaN,artwork,None,None,None,None,https://hudsonriverpark.org/activities/serpent...,None,None,None,None,None,None,None,None
2043,16231018,Vessel,NaN,NaN,artwork,None,None,None,None,https://www.hudsonyardsnewyork.com/discover/ve...,None,None,None,yes,None,None,Q27988215,en:Vessel (structure)
